# 卷积神经网络

## TensorFlow 中实现卷积神经网络

TensorFlow 提供了 `tf.nn.conv2d()` 和 `tf.nn.bias_add()` 函数来创建你自己的卷积层。

In [ ]:
# Output depth
k_output = 64

# Image Properties
image_width = 10
image_height = 10
color_channels = 3

# Convolution filter
filter_size_width = 5
filter_size_height = 5

# Input/Image
input = tf.placeholder(
    tf.float32,
    shape=[None, image_height, image_width, color_channels])

# Weight and bias
weight = tf.Variable(tf.truncated_normal(
    [filter_size_height, filter_size_width, color_channels, k_output]))
bias = tf.Variable(tf.zeros(k_output))

# Apply Convolution
conv_layer = tf.nn.conv2d(input, weight, strides=[1, 2, 2, 1], padding='SAME')
# Add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
# Apply activation function
conv_layer = tf.nn.relu(conv_layer)

上述代码用了 `tf.nn.conv2d()` 函数来计算卷积，weights 作为滤波器，`[1, 2, 2, 1]` 作为 strides。TensorFlow 对每一个 input 维度使用一个单独的 stride 参数，`[batch, input_height, input_width, input_channels]`。我们通常把 `batch` 和 `input_channels` （strides 序列中的第一个第四个）的 stride 设为 1。

你可以专注于修改 `input_height` 和 `input_width`， `batch` 和 `input_channels` 都设置成 1。`input_height` 和 `input_width` strides 表示滤波器在input 上移动的步长。上述例子中，在 input 之后，设置了一个 5x5 ，stride 为 2 的滤波器。

`tf.nn.bias_add()` 函数对矩阵的最后一维加了偏置项。

## 最大池化操作
![img](https://ws1.sinaimg.cn/large/69d4185bly1fpdb143kxoj20fu09674p.jpg)
例如 `[[1, 0], [4, 6]]` 生成 6，因为 6 是这4个数字中最大的。同理 `[[2, 3], [6, 8]]` 生成 8。 理论上，最大池化操作的好处是减小输入大小，使得神经网络能够专注于最重要的元素。最大池化只取覆盖区域中的最大值，其它的值都丢弃。

TensorFlow 提供了 `tf.nn.max_pool()` 函数，用于对卷积层实现 最大池化 。

In [ ]:
conv_layer = tf.nn.conv2d(input, weight, strides=[1, 2, 2, 1], padding='SAME')
conv_layer = tf.nn.bias_add(conv_layer, bias)
conv_layer = tf.nn.relu(conv_layer)
# Apply Max Pooling
conv_layer = tf.nn.max_pool(
    conv_layer,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='SAME')

`tf.nn.max_pool()` 函数实现最大池化时， `ksize`参数是滤波器大小，`strides`参数是步长。2x2 的滤波器配合 2x2 的步长是常用设定。

ksize 和 strides 参数也被构建为四个元素的列表，每个元素对应 input tensor 的一个维度 (`[batch, height, width, channels]`)，对 ksize 和 strides 来说，batch 和 channel 通常都设置成 1。

## TensorFlow 中的卷积网络
![img](https://ws1.sinaimg.cn/large/69d4185bly1fpdko13ps2g20em0aojsv.gif)
这是一个 3x3 的卷积滤波器的示例。以 stride 为 1 应用到一个范围在 0 到 1 之间的数据上。每一个 3x3 的部分与权值 `[[1, 0, 1], [0, 1, 0], [1, 0, 1]]` 做卷积，把偏置加上后得到右边的卷积特征。这里偏置是 0 。TensorFlow 中这是通过 tf.nn.conv2d() 和 tf.nn.bias_add() 来完成的。
```python
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)
```
`tf.nn.conv2d()`函数与权值 W 做卷积。

在 TensorFlow 中，strides 是一个4个元素的序列；第一个位置表示 stride 的 batch 参数，最后一个位置表示 stride 的特征(feature)参数。最好的移除 batch 和特征(feature)的方法是你直接在数据集中把他们忽略，而不是使用 stride。要使用所有的 batch 和特征(feature)，你可以把第一个和最后一个元素设成1。

中间两个元素指纵向(height)和横向(width)的 stride，之前也提到过 stride 通常是正方形，height = width。当别人说 stride 是 3 的时候，他们意思是 `tf.nn.conv2d(x, W, strides=[1, 3, 3, 1])`。

为了更简洁，这里的代码用了`tf.nn.bias_add()` 来添加偏置。 `tf.add()` 这里不能使用，因为 tensors 的维度不同。

### 数据集

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
# 参数
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
# 用来验证和计算准确率的样本数
# 如果内存不够，可以调小这个数字
test_valid_size = 256

# Network Parameters
# 神经网络参数
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


### 权重与偏差项

In [2]:

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

### 卷积

In [3]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

### 最大池化

In [4]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

### 模型
![img](https://ws1.sinaimg.cn/large/69d4185bly1fpdkupfax5j220210g7wh.jpg)

在下面的代码中，我们创建了 3 层来实现卷积，最大池化以及全链接层和输出层。每一层对维度的改变都写在注释里。例如第一层在卷积部分把图片从 28x28x1 变成了 28x28x32。后面应用了最大池化，每个样本变成了 14x14x32。从 conv1 经过多层网络，最后到 output 生成 10 个分类。

In [5]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

### session 中运行

In [7]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})
            if batch % 20 == 0:
                print('Epoch {:>2}, Batch {:>3} -'
                      'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                    epoch + 1,
                    batch + 1,
                    loss,
                    valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 49498.5742 Validation Accuracy: 0.132812
Epoch  1, Batch  21 -Loss: 15583.9990 Validation Accuracy: 0.355469
Epoch  1, Batch  41 -Loss: 10467.6045 Validation Accuracy: 0.519531
Epoch  1, Batch  61 -Loss:  6414.1133 Validation Accuracy: 0.601562
Epoch  1, Batch  81 -Loss:  5401.9346 Validation Accuracy: 0.648438
Epoch  1, Batch 101 -Loss:  3251.8738 Validation Accuracy: 0.683594
Epoch  1, Batch 121 -Loss:  3989.8267 Validation Accuracy: 0.675781
Epoch  1, Batch 141 -Loss:  3241.5171 Validation Accuracy: 0.722656
Epoch  1, Batch 161 -Loss:  2244.8250 Validation Accuracy: 0.726562
Epoch  1, Batch 181 -Loss:  2963.5161 Validation Accuracy: 0.753906
Epoch  1, Batch 201 -Loss:  3212.5054 Validation Accuracy: 0.769531
Epoch  1, Batch 221 -Loss:  2921.8550 Validation Accuracy: 0.773438
Epoch  1, Batch 241 -Loss:  1829.8149 Validation Accuracy: 0.761719
Epoch  1, Batch 261 -Loss:  2413.0225 Validation Accuracy: 0.773438
Epoch  1, Batch 281 -Loss:  2345.9712 Validation

Epoch  6, Batch 221 -Loss:   448.0390 Validation Accuracy: 0.871094
Epoch  6, Batch 241 -Loss:   456.3636 Validation Accuracy: 0.867188
Epoch  6, Batch 261 -Loss:   429.1323 Validation Accuracy: 0.867188
Epoch  6, Batch 281 -Loss:   479.3262 Validation Accuracy: 0.875000
Epoch  6, Batch 301 -Loss:   462.2725 Validation Accuracy: 0.871094
Epoch  6, Batch 321 -Loss:   591.1453 Validation Accuracy: 0.871094
Epoch  6, Batch 341 -Loss:   580.1155 Validation Accuracy: 0.863281
Epoch  6, Batch 361 -Loss:   632.9096 Validation Accuracy: 0.871094
Epoch  6, Batch 381 -Loss:   550.5165 Validation Accuracy: 0.867188
Epoch  6, Batch 401 -Loss:   664.2405 Validation Accuracy: 0.871094
Epoch  6, Batch 421 -Loss:   557.3735 Validation Accuracy: 0.867188
Epoch  7, Batch   1 -Loss:   601.0065 Validation Accuracy: 0.875000
Epoch  7, Batch  21 -Loss:   518.1102 Validation Accuracy: 0.875000
Epoch  7, Batch  41 -Loss:   390.7771 Validation Accuracy: 0.871094
Epoch  7, Batch  61 -Loss:   555.6370 Validation